#### Shows how the agent would play the game always choosing the best action

In [ ]:
def play_game(net, maze, pos, device):
    net.eval()
    maze.reset()
    maze.state = [pos[0], pos[1], maze.state[2]]
    envstate = maze.observe()
    counter = 0
    while True:
        counter += 1
        #time.sleep(0.4)
        prev_envstate = torch.Tensor(envstate).to(device)
        # get next action
        qvals = net(prev_envstate.view(1,1,20,20))
        action = torch.argmax(qvals)
        envstate, reward, game_status = maze.act(action)
        plt.imshow(maze.observe(empty=False).reshape(20,20))
        plt.show()
        #print('Move %i' %(counter))
        clear_output(wait = True)
        if game_status == 1:
            maze_show = copy.deepcopy(maze.observe(empty=False)).reshape(20,20)
            for cell in maze.visited:
                maze_show[cell] = 0.8
            maze_show[19,19] = 0.3
            fig = plt.figure(figsize=(12,12))
            plt.imshow(maze_show, cmap='gray')
            plt.xticks([])
            plt.yticks([])
            plt.show()
            fig.savefig('solved.pdf')
            return 1
        elif game_status == -1:
            return 0

#### Shows the flow of the maze

In [ ]:
def show_flow(net, maze, device):
    #0=left, 1=up, 2=right, 3=down
    maze_show = copy.deepcopy(maze.init_maze)
    for cell in [*maze.free_cells, (0,0)]:
        maze.state = (cell[0], cell[1], maze.state[2])
        maze_pos = torch.Tensor(maze.observe(empty=True)).to(device).view(1,1,20,20)
        qvals = net(maze_pos)
        action = torch.argmax(qvals)
        maze_show[cell] = action
    fig = plt.figure(figsize=(12,12))
    maze_plot = copy.deepcopy(maze.init_maze)
    maze_plot[0,0] = 1.0
    plt.imshow(maze_plot, cmap='gray')
    #plt.arrow(0,10,1,1,color='red', head_width=0.2)
    #plt.axis('off')
    plt.xticks([])
    plt.yticks([])
    COL_R = 'limegreen'
    COL_W = 'red'
    HW = 0.12
    LEN = 0.45
    wrong_cells = [(17,0), (17,1), (17,3), (17,4), (17,5), (18,0), (18,3), (18,4), (19,0), (19,1), (19,2),
                   (16,13), (16,14), (15,14), (19,18)]
    for cell in maze.free_cells:
        if cell in wrong_cells:
            COL = COL_W
        else:
            COL = COL_R
        if maze_show[cell]==0:
            plt.arrow(cell[1]+0.3, cell[0], -LEN, 0.0, color=COL, head_width=HW)
        if maze_show[cell]==1:
            plt.arrow(cell[1], cell[0]+0.3, 0.0, -LEN, color=COL, head_width=HW)
        if maze_show[cell]==2:
            plt.arrow(cell[1]-0.3, cell[0], LEN, 0.0, color=COL, head_width=HW)
        if maze_show[cell]==3:
            plt.arrow(cell[1], cell[0]-0.3, 0.0, LEN, color=COL, head_width=HW)
    plt.show()
    fig.savefig('flow.pdf')